In [ ]:
using DynamicalSystems
using CairoMakie
using IntervalArithmetic, IntervalRootFinding
using NLsolve
using ModelingToolkit
using NonlinearSolve

In [ ]:
function hillLikeFunction(HSL, K, n)
    """
    This function returns the value of a hill-like function
    Inputs:
        - K: Association constant
        - HSL: HSL concentration
        - n: Hill coefficient
    Returns:
        - The value of the hill-like function
    """
    return ((K * HSL)/(1 + K * HSL)) ^ n
    end


function promoterActivity(ACT, REP, K_TF_GENE, K_TF_HSL, K_DIMER, K_REP_GENE, c, n_TF, n_REP)
    """
    This function returns the value of the promoter function
    Inputs:
        - ACT: Activator concentration
        - REP: Repressor concentration
        - A: Activator concentration
        - R: Repressor concentration
        - K_TF_GENE: Association constant of the TF to the gene
        - K_TF_HSL: Association constant of the TF to the HSL
        - K_DIMER: Association constant of the TF dimerization
        - K_REP_GENE: Association constant of the repressor to the gene
        - c: HSL concentration
        - n_TF: Hill coefficient of the TF
        - n_REP: Hill coefficient of the repressor
    Returns:
        - The value of the promoter function
    """
    return (K_TF_GENE * K_DIMER * c^2 * hillLikeFunction(ACT, K_TF_HSL, n_TF)) / (
                1
                + K_TF_GENE * K_DIMER * c^2 * hillLikeFunction(ACT
                , K_TF_HSL, n_TF)
                + (K_REP_GENE*REP)^n_REP)
    end
    

In [ ]:
#function promoterActivity(A, R, K_TF_GENE, K_TF_HSL, K_REP_GENE, c, n_TF, n_REP)
    # Hill function for promoter activity
    # A: activator concentration
    # R: repressor concentration
    # K_TF_GENE: TF binding constant to gene
    # K_TF_HSL: TF binding constant to HSL
    # K_REP_GENE: repressor binding constant to gene
    # c: basal expression
    # n_TF: Hill coefficient for TF binding to gene
    # n_REP: Hill coefficient for repressor binding to gene
    # Translate the following to julia code:
    # (((K_TF_GENE**-1)*(1/0.02)*(c**2)*((K_TF_HSL*A)/(1 + K_TF_HSL*A))**n_TF)/(1 + (K_TF_GENE**-1)*(1/0.02)*(c**2)*((K_TF_HSL*A)/(1 + K_TF_HSL*A))**n_TF)) * (1/(1 + (R/K_REP_GENE)**n_REP))
#    return (((K_TF_GENE^-1)*(1/0.02)*(c^2)*((K_TF_HSL*A)/(1 + K_TF_HSL*A))^n_TF)/(1 + (K_TF_GENE^-1)*(1/0.02)*(c^2)*((K_TF_HSL*A)/(1 + K_TF_HSL*A))^n_TF)) * (1/(1 + (R/K_REP_GENE)^n_REP))
#end

In [ ]:
@inline @inbounds function simplifiedFerro(u, p, t)
    # C6i
    du0 = ((p[12] * u[7]) / (p[13] + u[7])) + p[15] * (u[3] - u[1]) - p[16] * u[1]
    # C12i
    du1 = ((p[12] * u[8]) / (p[13] + u[8])) + p[15] * (u[4] - u[2]) - p[16] * u[2]
    # C6e
    du2 = p[15] * (u[1] - u[3]) - p[17] * u[3]
    # C12e
    du3 = p[15] * (u[2] - u[4]) - p[17] * u[4]
    # TetR repressor
    du4 = (
        (promoterActivity(u[2], u[6], p[6], p[2], p[4], p[19], p[11], p[23], p[21])) * p[7]
        + p[25]
    ) * (p[8] / p[9]) - p[10] * u[5]
    # LacI repressor
    du5 = (
        (promoterActivity(u[1], u[5], p[5], p[1], p[3], p[18], p[11], p[22], p[20])) * p[7]
        + p[24]
    ) * (p[8] / p[9]) - p[10] * u[6]
    # LuxI enzyme
    du6 = (
        (promoterActivity(u[1], u[5], p[5], p[1], p[3], p[18], p[11], p[22], p[20])) * p[7]
        + p[24]
    ) * (p[8] / p[9]) - p[10] * u[7]
    # LasI enzyme
    du7 = (
        (promoterActivity(u[2], u[6], p[6], p[2], p[4], p[19], p[11], p[23], p[21])) * p[7]
        + p[25]
    ) * (p[8] / p[9]) - p[10] * u[8]
    # RFP (mCherry2)
    du8 = (
        (promoterActivity(u[1], u[5], p[5], p[1], p[3], p[18], p[11], p[22], p[20])) * p[7]
        + p[24]
    ) * (p[8] / p[9]) - p[10] * u[9]
    # GFP (sfGFP)
    du9 = (
        (promoterActivity(u[2], u[6], p[6], p[2], p[4], p[19], p[11], p[23], p[21])) * p[7]
        + p[25]
    ) * (p[8] / p[9]) - p[10] * u[10]

    return SVector{10}(du0, du1, du2, du3, du4, du5, du6, du7, du8, du9)
end

In [ ]:
# Define the parameters of the system in a list
p = [
    # VALUE # NAME # UNITS # DESCRIPTION
    # -------------------------------------------------------------
    # RATE PARAMETERS
    # -------------------------------------------------------------
    1/0.1,      # K_1R # uM^-1 # Equilibrium constant between LuxR and C6
    1/0.1,      # K_1S # uM^-1 # Equilibrium constant between LasR and C12
    1/0.02,     # K_2R # uM^-1 # Equilibrium constant between LuxR:C6 and C6
    1/0.02,     # K_2S # uM^-1 # Equilibrium constant between LasR:C12 and C12
    1/0.010,    # K_GR # uM^-1 # Equilibrium constant between LuxR:C6 and LuxR:C6:DNA
    1/0.010,    # K_GS # uM^-1 # Equilibrium constant between LasR:C12 and LasR:C12:DNA
    # -------------------------------------------------------------
    # TRANSCRIPTION AND TRANSLATION PARAMETERS
    # -------------------------------------------------------------
    1e-1,   # k_tx # uM/min # Constitutive transcription rate
    5e0,    # k_tr # min^-1 # Translation rate
    5e-1,   # mrna_deg # min^-1 # mRNA degradation rate
    5e-2,   # protein_deg # min^-1 # Protein degradation rate
    (1e-1 * 5e0)/(5e-1 * 5e-2), # QSSA # No units # Quasi-steady state assumption
    # -------------------------------------------------------------
    # LUXI AND LASI PARAMETERS
    # -------------------------------------------------------------
    0.04,   # prod_HSL # min^-1 # HSL production rate
    10.0,   # Km_HSL # uM # Michaelis-Menten constant of HSL production
    5e-2,   # d_HSL # min^-1 # HSL degradation rate
    # -------------------------------------------------------------
    # HSL parameters
    # -------------------------------------------------------------
    1.0e-1,           # Dm_HSL # uM # HSL diffusion constant
    0.001,             # d_HSLi # min^-1 # HSL degradation rate (intracellular)
    0.0001,            # d_HSLe # min^-1 # HSL degradation rate (extracellular)
    # -------------------------------------------------------------
    # Other Hill Function parameters
    # -------------------------------------------------------------
    1/0.001,#1/0.001,           # KR_TET # uM^-1 # Affinity constant of the tet repressor
    1/0.100,#1/0.015,           # KR_LAC # uM^-1 # Affinity constant of the lac repressor
    2,                          # n_TET # No units # Coopertivity of the TET repressor
    4,                          # n_LAC # No units # Coopertivity of the LAC repressor
    2,                          # n_LUX # No units # Coopertivity of the LUX activator
    2,                          # n_LAS # No units # Coopertivity of the LAS activator
    0.0050,                           # leak_LUX # uM/min # Leakiness of the LUX promoter
    0.0250,           # leak_LAS # uM/min # Leakiness of the LAS promoter
]

In [ ]:
# Initial state vector is defined as a list of 0
#u0 = [0.78, 2.4, 0.77, 2.38, 19.38, 2.73, 2.73, 19.38, 2.73, 19.38]
u0 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
# Create a continuous dynamical system
ds = ContinuousDynamicalSystem(simplifiedFerro, u0, p)

In [ ]:
# Compute the trajectory of the system
X, t = trajectory(ds, 1000)

In [ ]:
fig = Figure(resolution = (1000, 600))
ax = Axis(fig[1, 1]; xlabel = "time", ylabel = "variable")

labels = [
    "C6i" "C12i" "C6e" "C12e" "TetR" "LacI" "LuxI" "LasI" "mCherry2" "sfGFP"
]

counter = 1

for var in columns(X)
    lines!(ax, t, var, label = labels[counter])
    counter += 1
end

ax.limits = (0, nothing, 1e-4, 1e2)
ax.yscale = log10

# Show the legend
fig[1, 2] = Legend(fig, ax, "Chemical Species", framevisible = false)

fig

In [ ]:
# Compare the trajectories of mCherry2 and sfGFP
fig = Figure(resolution = (1000, 600))
ax = Axis(fig[1, 1]; xlabel = "time", ylabel = "variable")

labels = ["mCherry2" "sfGFP"]

counter = 1

for var in columns(X)
    if counter == 9 || counter == 10
        lines!(ax, t, var, label = labels[counter - 8])
    end
    counter += 1
end

ax.limits = (0, nothing, 1e-4, 1e2)
ax.yscale = log10

# Show the legend
fig[1, 2] = Legend(fig, ax, "Chemical Species", framevisible = false)

fig

In [ ]:
using Symbolics
# Define the system variables
@variables t C6e(t) C12e(t) C6i(t) C12i(t) TETR(t) LACI(t) LUXI(t) LASI(t) RFP(t) GFP(t)
# Define the system parameters
@variables K_1R K_1S K_2R K_2S K_2R K_2S K_GR K_GS k_tx k_tr d_m d_p QSSA prod_HSL Km_HSL d_HSL Dm_HSL d_HSLi d_HSLe KR_TET KR_LAC n_TET n_LAC n_LUX n_LAS leak_LUX leak_LAS
# Define the differential operator
D = Differential(t)
# Define the equations
eqns = [
    # C6i
    D(C6i) ~ ((prod_HSL * LUXI) / (Km_HSL + LUXI)) + Dm_HSL * (C6e - C6i) - d_HSLi * C6i
    # C12i
    D(C12i) ~ ((prod_HSL * LASI) / (Km_HSL + LASI)) + Dm_HSL * (C12e - C12i) - d_HSLi * C12i
    # C6e
    D(C6e) ~ Dm_HSL * (C6i - C6e) - d_HSLe * C6e
    # C12e
    D(C12e) ~ Dm_HSL * (C12i - C12e) - d_HSLe * C12e
    # TetR repressor
    D(TETR) ~ (
        (promoterActivity(C12i, LACI, K_GS, K_1S, K_2S, KR_LAC, QSSA, n_LAS, n_LAC)) * k_tx
        + leak_LAS
    ) * (k_tr / d_m) - d_p * TETR
    # LacI repressor
    D(LACI) ~ (
        (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R,KR_TET, QSSA, n_LUX, n_TET)) * k_tx
        + leak_LUX
    ) * (k_tr / d_m) - d_p * LACI
    # LuxI enzyme
    D(LUXI) ~ (
        (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, QSSA, n_LUX, n_TET)) * k_tx
        + leak_LUX
    ) * (k_tr / d_m) - d_p * LUXI
    # LasI enzyme
    D(LASI) ~ (
        (promoterActivity(C12i, LACI, K_GS, K_1S, K_2S, KR_LAC, QSSA, n_LAS, n_LAC)) * k_tx
        + leak_LAS
    ) * (k_tr / d_m) - d_p * LASI
    # RFP (mCherry2)
    D(RFP) ~ (
        (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, QSSA, n_LUX, n_TET)) * k_tx
        + leak_LUX
    ) * (k_tr / d_m) - d_p * RFP
    # GFP (sfGFP)
    D(GFP) ~ (
        (promoterActivity(C12i, LACI, K_GS, K_1S, K_2S, KR_LAC, QSSA, n_LAS, n_LAC)) * k_tx
        + leak_LAS
    ) * (k_tr / d_m) - d_p * GFP
]

In [ ]:
# Define the equations
eq_eqns = [
    # C6i
    0 ~ ((prod_HSL * LUXI) / (Km_HSL + LUXI)) + Dm_HSL * (C6e - C6i) - d_HSLi * C6i
    # C12i
    0 ~ ((prod_HSL * LASI) / (Km_HSL + LASI)) + Dm_HSL * (C12e - C12i) - d_HSLi * C12i
    # C6e
    0 ~ Dm_HSL * (C6i - C6e) - d_HSLe * C6e
    # C12e
    0 ~ Dm_HSL * (C12i - C12e) - d_HSLe * C12e
    # TetR repressor
    0 ~ (
        (promoterActivity(C12i, LACI, K_GS, K_1S, K_2S, KR_LAC, QSSA, n_LAS, n_LAC)) * k_tx
        + leak_LAS
    ) * (k_tr / d_m) - d_p * TETR
    # LacI repressor
    0 ~ (
        (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, QSSA, n_LUX, n_TET)) * k_tx
        + leak_LUX
    ) * (k_tr / d_m) - d_p * LACI
    # LuxI enzyme
    0 ~ (
        (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, QSSA, n_LUX, n_TET)) * k_tx
        + leak_LUX
    ) * (k_tr / d_m) - d_p * LUXI
    # LasI enzyme
    0 ~ (
        (promoterActivity(C12i, LACI, K_GS, K_1S, K_2S, KR_LAC, QSSA, n_LAS, n_LAC)) * k_tx
        + leak_LAS
    ) * (k_tr / d_m) - d_p * LASI
    # RFP (mCherry2)
    0 ~ (
        (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, QSSA, n_LUX, n_TET)) * k_tx
        + leak_LUX
    ) * (k_tr / d_m) - d_p * RFP
    # GFP (sfGFP)
    0 ~ (
        (promoterActivity(C12i, LACI, K_GS, K_1S, K_2S, KR_LAC, QSSA, n_LAS, n_LAC)) * k_tx
        + leak_LAS
    ) * (k_tr / d_m) - d_p * GFP
]

@named ns = NonlinearSystem(eq_eqns, [C6e, C12e, C6i, C12i, TETR, LACI, LUXI, LASI, RFP, GFP], [K_1R, K_1S, K_2R, K_2S, K_GR, K_GS, k_tx, k_tr, d_m, d_p, QSSA, prod_HSL, Km_HSL, d_HSL, Dm_HSL, d_HSLi, d_HSLe, KR_TET, KR_LAC, n_TET, n_LAC, n_LUX, n_LAS, leak_LUX, leak_LAS])

In [ ]:
guess = [
    # C6e
    1e-1
    # C12e
    1e-6
    # C6i
    1e-1
    # C12i
    1e-6
    # TetR repressor
    1e-6
    # LacI repressor
    1e-1
    # LxI enzyme
    1e-1
    # LasI enzyme
    1e-1
    # RFP (mCherry2)
    1e-1
    # GFP (sfGFP)
    1e-6
]

params = [
    # VALUE # NAME # UNITS # DESCRIPTION
    # -------------------------------------------------------------
    # RATE PARAMETERS
    # -------------------------------------------------------------
    K_1R => 1/0.1,      # K_1R # uM^-1 # Equilibrium constant between LuxR and C6
    K_1S => 1/0.1,      # K_1S # uM^-1 # Equilibrium constant between LasR and C12
    K_2R => 1/0.02,     # K_2R # uM^-1 # Equilibrium constant between LuxR:C6 and C6
    K_2S => 1/0.02,     # K_2S # uM^-1 # Equilibrium constant between LasR:C12 and C12
    K_GR => 1/0.010,    # K_GR # uM^-1 # Equilibrium constant between LuxR:C6 and LuxR:C6:DNA
    K_GS => 1/0.010,    # K_GS # uM^-1 # Equilibrium constant between LasR:C12 and LasR:C12:DNA
    # -------------------------------------------------------------
    # TRANSCRIPTION AND TRANSLATION PARAMETERS
    # -------------------------------------------------------------
    k_tx => 1e-1,   # k_tx # uM/min # Constitutive transcription rate
    k_tr => 5e0,    # k_tr # min^-1 # Translation rate
    d_m => 5e-1,   # mrna_deg # min^-1 # mRNA degradation rate
    d_p => 5e-2,   # protein_deg # min^-1 # Protein degradation rate
    QSSA => (1e-1 * 5e0)/(5e-1 * 5e-2), # QSSA # No units # Quasi-steady state assumption
    # -------------------------------------------------------------
    # LUXI AND LASI PARAMETERS
    # -------------------------------------------------------------
    prod_HSL => 0.04,   # prod_HSL # min^-1 # HSL production rate
    Km_HSL => 10.0,   # Km_HSL # uM # Michaelis-Menten constant of HSL production
    d_HSL => 5e-2,   # d_HSL # min^-1 # HSL degradation rate
    # -------------------------------------------------------------
    # HSL parameters
    # -------------------------------------------------------------
    Dm_HSL => 1.0e-1,           # Dm_HSL # uM # HSL diffusion constant
    d_HSLi => 0.001,             # d_HSLi # min^-1 # HSL degradation rate (intracellular)
    d_HSLe => 0.0001,            # d_HSLe # min^-1 # HSL degradation rate (extracellular)
    # -------------------------------------------------------------
    # Other Hill Function parameters
    # -------------------------------------------------------------
    KR_TET => 1/0.001,#1/0.001,           # KR_TET # uM^-1 # Affinity constant of the tet repressor
    KR_LAC => 1/0.100,#1/0.015,           # KR_LAC # uM^-1 # Affinity constant of the lac repressor
    n_TET => 2,                          # n_TET # No units # Coopertivity of the TET repressor
    n_LAC => 4,                          # n_LAC # No units # Coopertivity of the LAC repressor
    n_LUX => 2,                          # n_LUX # No units # Coopertivity of the LUX activator
    n_LAS => 2,                          # n_LAS # No units # Coopertivity of the LAS activator
    leak_LUX => 5e-5,                           # leak_LUX # uM/min # Leakiness of the LUX promoter
    leak_LAS => 2.5e-4,           # leak_LAS # uM/min # Leakiness of the LAS promoter
]

prob = NonlinearProblem(ns, guess, params, jac=true)
sol = solve(prob, NewtonRaphson())

# DifferentialEquations solving for steady state

In [ ]:
function ODEsimplifiedFerro!(du, u, p, t)
    # C6i
    du[1] = ((p[12] * u[7]) / (p[13] + u[7])) + p[15] * (u[3] - u[1]) - p[16] * u[1]
    # C12i
    du[2] = ((p[12] * u[8]) / (p[13] + u[8])) + p[15] * (u[4] - u[2]) - p[16] * u[2]
    # C6e
    du[3] = p[15] * (u[1] - u[3]) - p[17] * u[3]
    # C12e
    du[4] = p[15] * (u[2] - u[4]) - p[17] * u[4]
    # TetR repressor
    du[5] = (
        (promoterActivity(u[2], u[6], p[6], p[2], p[4], p[19], p[11], p[23], p[21])) * p[7]
        + p[25]
    ) * (p[8] / p[9]) - p[10] * u[5]
    # LacI repressor
    du[6] = (
        (promoterActivity(u[1], u[5], p[5], p[1], p[3], p[18], p[11], p[22], p[20])) * p[7]
        + p[24]
    ) * (p[8] / p[9]) - p[10] * u[6]
    # LuxI enzyme
    du[7] = (
        (promoterActivity(u[1], u[5], p[5], p[1], p[3], p[18], p[11], p[22], p[20])) * p[7]
        + p[24]
    ) * (p[8] / p[9]) - p[10] * u[7]
    # LasI enzyme
    du[8] = (
        (promoterActivity(u[2], u[6], p[6], p[2], p[4], p[19], p[11], p[23], p[21])) * p[7]
        + p[25]
    ) * (p[8] / p[9]) - p[10] * u[8]
    # RFP (mCherry2)
    du[9] = (
        (promoterActivity(u[1], u[5], p[5], p[1], p[3], p[18], p[11], p[22], p[20])) * p[7]
        + p[24]
    ) * (p[8] / p[9]) - p[10] * u[9]
    # GFP (sfGFP)
    du[10] = (
        (promoterActivity(u[2], u[6], p[6], p[2], p[4], p[19], p[11], p[23], p[21])) * p[7]
        + p[25]
    ) * (p[8] / p[9]) - p[10] * u[10]
end

In [ ]:
using DifferentialEquations

In [ ]:
u0 = [
    # C6i
    1e-3;
    # C12i
    0;
    # C6e
    0;
    # C12e
    0;
    # TetR repressor
    0;
    # LacI repressor
    0;
    # LuxI enzyme
    0;
    # LasI enzyme
    0;
    # RFP (mCherry2)
    0;
    # GFP (sfGFP)
    0;
]

# Define the parameters of the system in a list
p = [
    # VALUE # NAME # UNITS # DESCRIPTION
    # -------------------------------------------------------------
    # RATE PARAMETERS
    # -------------------------------------------------------------
    1/0.1,      # K_1R # uM^-1 # Equilibrium constant between LuxR and C6
    1/0.1,      # K_1S # uM^-1 # Equilibrium constant between LasR and C12
    1/0.02,     # K_2R # uM^-1 # Equilibrium constant between LuxR:C6 and C6
    1/0.02,     # K_2S # uM^-1 # Equilibrium constant between LasR:C12 and C12
    1/0.010,    # K_GR # uM^-1 # Equilibrium constant between LuxR:C6 and LuxR:C6:DNA
    1/0.010,    # K_GS # uM^-1 # Equilibrium constant between LasR:C12 and LasR:C12:DNA
    # -------------------------------------------------------------
    # TRANSCRIPTION AND TRANSLATION PARAMETERS
    # -------------------------------------------------------------
    1e-1,   # k_tx # uM/min # Constitutive transcription rate
    5e0,    # k_tr # min^-1 # Translation rate
    5e-1,   # mrna_deg # min^-1 # mRNA degradation rate
    5e-2,   # protein_deg # min^-1 # Protein degradation rate
    (1e-1 * 5e0)/(5e-1 * 5e-2), # QSSA # No units # Quasi-steady state assumption
    # -------------------------------------------------------------
    # LUXI AND LASI PARAMETERS
    # -------------------------------------------------------------
    0.04,   # prod_HSL # min^-1 # HSL production rate
    10.0,   # Km_HSL # uM # Michaelis-Menten constant of HSL production
    5e-2,   # d_HSL # min^-1 # HSL degradation rate
    # -------------------------------------------------------------
    # HSL parameters
    # -------------------------------------------------------------
    1.0e-1,           # Dm_HSL # uM # HSL diffusion constant
    0.01,             # d_HSLi # min^-1 # HSL degradation rate (intracellular)
    0.001,            # d_HSLe # min^-1 # HSL degradation rate (extracellular)
    # -------------------------------------------------------------
    # Other Hill Function parameters
    # -------------------------------------------------------------
    1/0.001,#1/0.001,           # KR_TET # uM^-1 # Affinity constant of the tet repressor
    1/0.100,#1/0.015,           # KR_LAC # uM^-1 # Affinity constant of the lac repressor
    2,                          # n_TET # No units # Coopertivity of the TET repressor
    4,                          # n_LAC # No units # Coopertivity of the LAC repressor
    2,                          # n_LUX # No units # Coopertivity of the LUX activator
    2,                          # n_LAS # No units # Coopertivity of the LAS activator
    5e-5,                           # leak_LUX # uM/min # Leakiness of the LUX promoter
    2.5e-4,           # leak_LAS # uM/min # Leakiness of the LAS promoter
]

tspan = (0.0, 1440.0)

prob = ODEProblem(ODEsimplifiedFerro!, u0, tspan, p)

In [ ]:
# Now define a SteadyStateProblem
ssprob = SteadyStateProblem(prob)

In [ ]:
sol = solve(ssprob, DynamicSS(Rodas5()))

In [ ]:
# Create a loop for finding different steady states by changing the initial conditions of C6e
# We will use the parameters defined above
# Define the initial conditions of the system
u0 = [
    # C6i
    0.0;
    # C12i
    0.0;
    # C6e
    0.0;
    # C12e
    0.0;
    # TetR repressor
    0.0;
    # LacI repressor
    0.0;
    # LuxI enzyme
    0.0;
    # LasI enzyme
    0.0;
    # RFP (mCherry2)
    0.0;
    # GFP (sfGFP)
    0.0;
]

# Create a linear space of 1000 points between 1e-4 and 1e-1
c1 = range(1e-5, 1e-4, length = 100)
c2 = range(1e-4, 1e-3, length = 100)
c3 = range(1e-3, 1e-2, length = 100)
c4 = range(1e-2, 1e-1, length = 100)
c5 = range(1e-1, 1e0, length = 100)
C6e = vcat(c1, c2, c3, c4, c5)
C6e = unique(C6e)

# Now create a meshgrid of the same C6e and C12e values
C12e = C6e

MESH = collect(Iterators.product(C6e, C12e))

# Create a list to store the steady state values of the system
ss = []

# Create the for loop
for i in MESH
    # Set the initial condition of C6e and C12e
    u0[3] = i[1]
    u0[4] = i[2]
    # Create the ODEProblem
    prob = ODEProblem(ODEsimplifiedFerro!, u0, tspan, p)
    # Create the SteadyStateProblem
    ssprob = SteadyStateProblem(prob)
    # Solve the SteadyStateProblem
    sol = solve(ssprob, DynamicSS(Rodas5()))
    # Round the steady state values to the 3rd decimal place
    sol = round.(sol, digits = 2)
    # Append the steady state values to the list
    push!(ss, sol)
end


In [ ]:
# Get the unique steady state values
ss = unique(ss)

# BifurcationKit

In [ ]:
using Revise, Parameters
using BifurcationKit
const BK = BifurcationKit

In [ ]:
function simplifiedFerroBif!(u, p)

    # Unpack the parameters
    @unpack K_1R, K_1S, K_2R, K_2S, K_GR, K_GS, k_tx, k_tr, d_m, d_p, prod_HSL, Km_HSL, d_HSL, Dm_HSL, d_HSLi, d_HSLe, KR_TET, KR_LAC, n_TET, n_LAC, n_LUX, n_LAS, leak_LUX, leak_LAS = p
    # Unpack the variables
    C6i, C12i, C6e, C12e, TETR, LACI, LUXI, LASI, RFP, GFP = u

    [
        # C6i
        ((prod_HSL * LUXI) / (Km_HSL + LUXI)) + Dm_HSL * (C6e - C6i) - d_HSLi * C6i,
        # C12i
        ((prod_HSL * LASI) / (Km_HSL + LASI)) + Dm_HSL * (C12e - C12i) - d_HSLi * C12i,
        # C6e
        Dm_HSL * (C6i - C6e) - d_HSLe * C6e,
        # C12e
        Dm_HSL * (C12i - C12e) - d_HSLe * C12e,
        # TetR repressor
        (
            (promoterActivity(C12i, LACI, K_GS, K_1R, K_2R, KR_LAC, (k_tr * k_tx)/(d_m * d_p), n_LUX, n_LAC)) * k_tx
            + leak_LAS
        ) * (k_tr / d_m) - d_p * TETR,
        # LacI repressor
        (
            (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, (k_tr * k_tx)/(d_m * d_p), n_LUX, n_TET)) * k_tx
            + leak_LUX
        ) * (k_tr / d_m) - d_p * LACI,
        # LuxI enzyme
        (
            (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, (k_tr * k_tx)/(d_m * d_p), n_LUX, n_TET)) * k_tx
            + leak_LUX
        ) * (k_tr / d_m) - d_p * LUXI,
        # LasI enzyme
        (
            (promoterActivity(C12i, LACI, K_GS, K_1R, K_2R, KR_LAC, (k_tr * k_tx)/(d_m * d_p), n_LUX, n_LAC)) * k_tx
            + leak_LAS
        ) * (k_tr / d_m) - d_p * LASI,
        # RFP (mCherry2)
        (
            (promoterActivity(C6i, TETR, K_GR, K_1R, K_2R, KR_TET, (k_tr * k_tx)/(d_m * d_p), n_LUX, n_TET)) * k_tx
            + leak_LUX
        ) * (k_tr / d_m) - d_p * RFP,
        # GFP (sfGFP)
        (
            (promoterActivity(C12i, LACI, K_GS, K_1R, K_2R, KR_LAC, (k_tr * k_tx)/(d_m * d_p), n_LUX, n_LAC)) * k_tx
            + leak_LAS
        ) * (k_tr / d_m) - d_p * GFP
    ]
end

bif_params = (
    # VALUE # NAME # UNITS # DESCRIPTION
    # -------------------------------------------------------------
    # RATE PARAMETERS
    # -------------------------------------------------------------
    K_1R = 1/0.1,      # K_1R # uM^-1 # Equilibrium constant between LuxR and C6
    K_1S = 1/0.1,      # K_1S # uM^-1 # Equilibrium constant between LasR and C12
    K_2R = 1/0.02,     # K_2R # uM^-1 # Equilibrium constant between LuxR:C6 and C6
    K_2S = 1/0.02,     # K_2S # uM^-1 # Equilibrium constant between LasR:C12 and C12
    K_GR = 1/0.010,    # K_GR # uM^-1 # Equilibrium constant between LuxR:C6 and LuxR:C6:DNA
    K_GS = 1/0.010,    # K_GS # uM^-1 # Equilibrium constant between LasR:C12 and LasR:C12:DNA
    # -------------------------------------------------------------
    # TRANSCRIPTION AND TRANSLATION PARAMETERS
    # -------------------------------------------------------------
    k_tx = 1e-1,   # k_tx # uM/min # Constitutive transcription rate
    k_tr = 5e0,    # k_tr # min^-1 # Translation rate
    d_m = 5e-1,   # mrna_deg # min^-1 # mRNA degradation rate
    d_p = 5e-2,   # protein_deg # min^-1 # Protein degradation rate
    #QSSA = (1e-1 * 5e0)/(5e-1 * 5e-2), # QSSA # No units # Quasi-steady state assumption
    # -------------------------------------------------------------
    # LUXI AND LASI PARAMETERS
    # -------------------------------------------------------------
    prod_HSL = 0.04,   # prod_HSL # min^-1 # HSL production rate
    Km_HSL = 10.0,   # Km_HSL # uM # Michaelis-Menten constant of HSL production
    d_HSL = 5e-2,   # d_HSL # min^-1 # HSL degradation rate
    # -------------------------------------------------------------
    # HSL parameters
    # -------------------------------------------------------------
    Dm_HSL = 1e-1,           # Dm_HSL # uM # HSL diffusion constant
    d_HSLi = 0.01,             # d_HSLi # min^-1 # HSL degradation rate (intracellular)
    d_HSLe = 0.001,            # d_HSLe # min^-1 # HSL degradation rate (extracellular)
    # -------------------------------------------------------------
    # Other Hill Function parameters
    # -------------------------------------------------------------
    KR_TET = 1/0.001,#1/0.001,           # KR_TET # uM^-1 # Affinity constant of the tet repressor
    KR_LAC = 1/0.100,#1/0.015,           # KR_LAC # uM^-1 # Affinity constant of the lac repressor
    n_TET = 2.0,                          # n_TET # No units # Coopertivity of the TET repressor
    n_LAC = 4.0,                          # n_LAC # No units # Coopertivity of the LAC repressor
    n_LUX = 2.0,                          # n_LUX # No units # Coopertivity of the LUX activator
    n_LAS = 2.0,                          # n_LAS # No units # Coopertivity of the LAS activator
    leak_LUX = 5e-5,                           # leak_LUX # uM/min # Leakiness of the LUX promoter
    leak_LAS = 2.5e-4,           # leak_LAS # uM/min # Leakiness of the LAS promoter
)

# Initial conditions
#u0 = [1.82, 0.02, 1.8, 0.02, 0.05, 9.98, 9.98, 0.05, 8.0, 0.05]
u0 = [2.43, 0.02, 2.4, 0.02, 0.05, 19.98, 19.98, 0.05, 19.98, 0.05]

# bifurcation problem C6i = u[1], C12i = u[2], C6e = u[3], C12e = u[4], TetR = u[5], LacI = u[6], LuxI = u[7], LasI = u[8],
recordFromSolutionFerro(u, p) = (mCherry2 = u[9], sfGFP = u[10])
#recordFromSolutionFerro(u, p) = (C6i = u[1], C12i = u[2])
# Create the bifurcation Problem
bif_prob = BifurcationProblem(
    simplifiedFerroBif!,
    u0,
    bif_params,
    (@lens _.KR_LAC);
    record_from_solution = recordFromSolutionFerro)

In [ ]:
# continuation options
opt_newton = NewtonPar(tol = 1e-8, max_iterations = 10000)

opts_br = ContinuationPar(p_min = 1e-4, p_max = 1e5,
	# parameters to have a smooth continuation curve
	ds = -0.01, dsmax = 0.1, dsmin = 0.0001, max_steps=1e6, detect_fold=true, detect_bifurcation = 3)

# continuation of equilibria
br = BK.continuation(bif_prob, PALC(tangent=Bordered()), opts_br; normC = norminf)

In [ ]:
br.sol

In [ ]:
using Plots
const PL = Plots

In [ ]:
PL.plot(br, xaxis=:log10, yaxis=:log10)

In [ ]:
col = [stb ? :black : :lightgrey for stb in br.stable]
# Do the same for linestyles
linestyle = [stb ? :solid : :solid for stb in br.stable]
# And for the linewidths
linewidth = [stb ? 1.5 : 1.0 for stb in br.stable]
PL.plot(br, color=col, linestyle=linestyle, linewidth=linewidth, xaxis=:log10, yaxis=:log10)
#PL.plot(br, yaxis=:log10, xaxis=:log10)
#savefig("bifurcation_k_tr.pdf")

In [ ]:
get_normal_form(br, 1;
	verbose = false, ζs = nothing, lens = getlens(br))